# **Change Request Risk Assessment with Cortex AI**
#### Author: **John Heisler** - Senior AI Specialist, Financial Services, Snowflake
In this notebook, we're going to evaluate a new change request and determine a risk score for that change request resulting in target environment instability.

The notebook is written to allow deployment directly into anyone's environment.

## Cortex AI Commercial Value

Ultimately, AI should drive commercial value. To that end, building, deploying, and maintaining the systems that underpin that commercial value need to be easy, efficient, and trusted. 

With Cortex AI, we balance complexity encapsulation with complexity exposure to maximize the time to value with AI. 

### Hit the Nail, Don't Build the Hammer!
Enterprises that maximize efforts leveraging their differentiated domain knowledge and creativity will ultimately win the AI race. Every ounce of effort spent on managing/building/tuning AI is a distraction from delivering value and thus comes at a material opportunity cost. With Snowflake Cortex AI, we aim to maximize efforts wielding the power of AI, not building it.

## Use Case Commercial Value
Our solution drives **operational alpha** by maximizing uptime of production environments. This solution provides operational alpha in at least these four ways:
1. **System Uptime**: The systems critical to support our portfolio teams will be more stable and provide maximum value to our portfolio teams' performance.
2. **Regulatory Reporting**: Minimize need for reporting to external regulators about critical system downtime and its impact.
3. **Opportunity Cost**: Minimizing time spent conducting emergency maintenance which can be repurposed to focus on alpha-generating solutions.
4. **Opportunity Cost**: Minimize need for time-and-resource-intensive root cause analysis and these cross functional teams can focus on their primary responsibilities.

## Process Outline
First, we will create and fill a change request table with some synthetic data (Fun Fact: You can use LLMs to do that. Check out how I did it in harbinger_data_creation.ipynb notebook in this repo). 

Second, we will build a python function to house our prompt and accept a change request as context. this approach streamlines our code and decouples the prompt from our broader development, allowing for independent development on the prompt by domain experts.

Last, we will build a Streamlit in Snowflake (SiS) front end to allow our end users to score new change requests.

## Snowflake Differentiators
* **LLM fungibility**: We have a model garden right here in Snowflake-- no need to manage divergent infrastructure and no need for external calls which introduce risk into your system
* **Physics of data**: With snowflake, we can perform the inference with LLMs all in one spot. There is no need to dehydrate Snowflake and move externally 
* **Flexibility**: Snowpark Container Services would allow you to bring any model you want (or any functionality for that matter) to run right here in Snowflake. This opens the door to run anything in Snowflake to maximize performance, minimize overhead, and reap the benefits of a single governance framework.
* **Access to Power**: We provide access to GPUs and infrastructure at your fingertips to turbo charge your development and performance. Again, this allows you to focus on driving value not button booping and knob twisting. 

## Art of the Possible
Here is some food for thought and hopefully inspiring enhancements for your deployment. 

* **Fine Tuning**: If we have tied incidents' root causes to change requests in the past, we could fine tune a model here in Snowflake with that data and the task that we're after. This has a couple of very interesting advantages. first, we would have a very specialize model that may perform this task very well because it has "experience" with what good and bad looks like. More operationally focused, we could maybe use a smaller model maximizing cost efficiencies.
* **More Context - Metadata**: we could offer much more context about what each column in the data set means and fully define the json structure that we're passing. This would inform the model on the meaning of each column rather than allowing it to come up with its best guess at the columns.
* **More Context - Target System Stability (Windshield)**: We could use another LLM upstream of the final risk inference to build a synopsis of the target system stability and status. For instance, it could state things like, "the target system for EDM-Account-Master has seen several out of memory alerts and disc space errors in the last six weeks." We would instruct the risk prompt to consider that environment when deriving its risk assessment which should enhance its predictions.
* **More Context - Incident Root Causes (Rear view Mirror)**: We could pass a synopsis of the last 6 months of root causes to the model. This would make it keenly aware 

## Next Steps
1. Try this out on your own data: You will be *blown away* by the results and how easy it is. 
2. Don't walk alone: You have access to AI experts that can guide your development to an accelerated outcome at no cost to you!

# Create and Load Change Request Table

In [ ]:
CREATE OR REPLACE TABLE ChangeRequests (
       ChangeNumber VARCHAR(100),
       Description VARCHAR(16777216),
       Date DATE,
       Impact VARCHAR(4000),
       Priority VARCHAR(400),
       Risk VARCHAR(400),
       Justification VARCHAR(4000),
       State VARCHAR(400),
       Disposition VARCHAR(400),
       Category VARCHAR(400)
   );

INSERT INTO ChangeRequests (ChangeNumber, Description, Date, Impact, Priority, Risk, Justification, State, Disposition, Category) VALUES
   ('AB1672', 'This change request proposes an upgrade to the existing DataSync API from version 3.2 to 3.4 in the Production environment. The update includes several key performance optimizations, enhanced security features, and bug fixes that address issues with data consistency and processing time. The major components of this change include: API Version Update: Migrating from DataSync API v3.2 to v3.4 to support faster data ingestion and processing. Security Enhancements: Implementation of OAuth 2.0-based authentication to replace the legacy basic authentication mechanism, improving overall security for API transactions. Error Handling: Enhanced error codes and more descriptive responses for improved debugging in the event of failure. Database Schema Update: Modifications to the backend MySQL database to accommodate new data types introduced in version 3.4. Testing will be performed in the staging environment (v3.4-Stage) before deployment to ensure backwards compatibility with existing systems. No downtime is expected during the deployment, but a rollback plan has been prepared in case of critical issues.', '2024-11-11', 'Improved performance and security with minimal downtime', 'immediately', 'low risk', 'preventative maintenance', 'new', 'no peer review', 'planned'),
   ('723CS', 'This change request outlines modifications to the current load balancer configuration in the production environment. The goal is to optimize traffic distribution across the primary application servers (AppSrv-01, AppSrv-02, AppSrv-03) to improve system reliability during high traffic spikes. Current Setup: The load balancer (LB-Prod) is currently set to a round-robin algorithm with a static weight of 1 for each server. Proposed Change: Implement a dynamic load balancing algorithm using least connections for improved resource distribution based on real-time traffic. Configuration Details: Modify the load balancing strategy from round-robin to Least Connections for AppSrv-01, AppSrv-02, and AppSrv-03. Increase the weight of AppSrv-03 (currently at weight 1) to weight 2, as it is the most powerful server. Implement automatic failover configuration in the event of server downtime to reduce service interruptions. Adjust session persistence settings for clients with long transactions to be sticky to the same backend server. Testing will be conducted on a staging environment (LB-Staging) to ensure optimal performance before full deployment. Expected downtime is minimal and will occur during the initial configuration phase.', '2024-11-11', 'Improved system reliability with minimal downtime', 'within release window', 'moderate risk', 'patch', 'under review', 'manager override', 'unplanned'),
   ('6D62EE', 'This change request aims to reconfigure the existing MySQL database cluster (DBCluster-Prod) to support horizontal scaling by introducing a sharded database model. This will help improve read and write throughput, especially for the rapidly growing user base in the e-commerce application. Current Setup: The current setup consists of a single master node (DBMaster-Prod) and two read replicas (DBReplica1, DBReplica2). Proposed Change: Introduce horizontal sharding by partitioning the user data across four separate database instances: DBShard-01, DBShard-02, DBShard-03, DBShard-04. Implement MySQL ProxySQL to handle routing of read and write queries to the appropriate shard based on the user’s region. Set up automatic sharding management to allow for easy redistribution of data as new shards are added in the future. Deploy a Data Sync Tool for initial data migration from DBMaster-Prod to the new shards. Update application logic to support sharded database queries, ensuring that application components such as user authentication and order management are correctly routed to the right database shards. Testing will be conducted in a dedicated staging environment (DBCluster-Staging) to verify data integrity and application performance. A backup of all critical data will be taken before the migration begins. Expected downtime is approximately 30 minutes for initial shard setup.', '2024-11-11', 'Improved read and write throughput with minimal downtime', 'after approval', 'high risk', 'planned release as part of project koala', 'approved', 'uniform agreement', 'emergency'),
 ('AB1673', 'This change request proposes a server migration from the legacy physical servers (Srv-Old-01, Srv-Old-02) to a new virtualized environment in VMware vSphere. The migration will improve resource allocation, reduce hardware costs, and provide better scalability. The new setup will include 4 VMs (VM-01 to VM-04) configured for redundancy and load balancing. Initial migration testing will be done on the staging environment (VMware-Staging), with a 2-hour maintenance window required for the final migration.', '2024-11-11', 'Improved scalability and cost savings', 'within maintenance window', 'moderate risk', 'cost optimization', 'approved', 'no peer review', 'planned'),
   ('AB1674', 'This change request outlines the integration of the new Customer Relationship Management (CRM) tool into the existing infrastructure. The CRM tool, Salesforce Cloud, will replace the legacy in-house system. The integration will involve data migration, API connections to internal systems, and user training. The migration process will begin in the staging environment, followed by a phased rollout to the production environment.', '2024-11-11', 'Enhanced customer interaction and reporting capabilities', 'within release window', 'low risk', 'system replacement', 'new', 'peer-reviewed', 'planned'),
   ('AB1675', 'This change request involves patching the security vulnerabilities in the web application firewall (WAF) configuration. The update will address critical vulnerabilities identified in the recent security audit. Changes will include tightening rules for SQL injection and cross-site scripting (XSS), and enhancing logging for monitoring suspicious activity. Testing will be done in the staging environment before pushing the update live in production.', '2024-11-11', 'Enhanced security of web applications', 'immediately', 'high risk', 'security patch', 'under review', 'manager override', 'unplanned'),
   ('AB1676', 'This change request proposes the upgrade of the Apache web server from version 2.4.46 to 2.4.54. The upgrade includes performance improvements, security patches, and support for HTTP/3. Additionally, several deprecated modules will be removed, and new SSL/TLS certificates will be installed. No downtime is expected, but a rollback plan is in place for potential issues.', '2024-11-11', 'Improved performance and security', 'within release window', 'low risk', 'system upgrade', 'new', 'no peer review', 'planned'),
   ('AB1677', 'This change request involves expanding the cloud storage capacity on AWS S3 by 5 TB to accommodate growing user data from the mobile app. The expansion will be managed using an automated script to ensure consistent provisioning, and backups will be taken before initiating the process. The expansion is expected to be seamless with minimal user impact.', '2024-11-11', 'Increased storage capacity with no downtime', 'within maintenance window', 'low risk', 'storage expansion', 'approved', 'peer-reviewed', 'unplanned'),
   ('AB1678', 'This change request suggests a reconfiguration of the DNS settings for the company’s public-facing websites. The objective is to improve redundancy and performance by integrating Route 53 with health checks and failover routing. The change will be tested in a sandbox environment before being pushed live to avoid downtime during the migration.', '2024-11-11', 'Improved availability and performance', 'within release window', 'moderate risk', 'performance optimization', 'under review', 'no peer review', 'planned'),
   ('AB1679', 'This change request proposes a new backup solution using Veeam Backup & Replication. The current backup system is inefficient and requires manual intervention. The new solution will automate backup jobs, include replication to an offsite location, and implement encryption for secure data storage. The initial setup and testing will occur in the staging environment before going live.', '2024-11-11', 'Automated backups with enhanced security', 'within maintenance window', 'low risk', 'new system installation', 'approved', 'manager override', 'planned'),
   ('AB1680', 'This change request involves the migration of the internal email system from Exchange Server 2016 to Microsoft 365. This migration will enable enhanced collaboration features, improve user experience, and reduce on-premises maintenance. The migration will be carried out in stages, starting with a pilot group, and full deployment will occur over the course of 2 weeks. A fallback plan is in place in case of service disruption.', '2024-11-11', 'Improved email functionality and collaboration tools', 'after approval', 'moderate risk', 'service migration', 'new', 'peer-reviewed', 'unplanned'),
   ('AB1681', 'This change request aims to upgrade the enterprise network infrastructure by replacing the aging core switches with Cisco Catalyst 9300 models. The upgrade will enhance network speed, support higher throughput, and improve security features such as 802.1X authentication. The work will be scheduled for a weekend to minimize downtime and will involve several phases of testing.', '2024-11-11', 'Improved network performance and security', 'within release window', 'high risk', 'infrastructure upgrade', 'approved', 'uniform agreement', 'planned'),
   ('AB1682', 'This change request proposes to integrate the payment gateway API (Stripe) into the company’s e-commerce platform to replace the legacy payment processor. The integration will include testing for PCI DSS compliance, customer notification, and system load testing to ensure performance during peak traffic periods. The rollout will happen incrementally, with extensive monitoring during the first week.', '2024-11-11', 'Enhanced payment processing capabilities', 'immediately', 'moderate risk', 'service upgrade', 'new', 'no peer review', 'unplanned'),
('AB1673', 'This change request proposes the migration of internal file storage from an on-premises SAN (Storage Area Network) to Amazon S3 for better scalability and cost optimization. The process will involve the migration of over 10 TB of data using AWS DataSync, with a staged migration plan that includes data validation, checksum comparisons, and testing to ensure integrity. The new solution will leverage S3 lifecycle policies for automated archival and deletion, and a CloudFront CDN will be configured for faster content delivery. The migration will happen in batches, with a 48-hour downtime window per batch, during which all file-related services will be unavailable.', '2024-11-11', 'Enhanced scalability and reduced operational cost', 'within release window', 'moderate risk', 'storage migration', 'approved', 'manager override', 'unplanned'),
   ('AB1674', 'This change request involves the reconfiguration of the Kubernetes cluster running in Google Kubernetes Engine (GKE). The changes will include increasing the number of nodes in the cluster from 5 to 15 to meet the increasing load of microservices. The configuration will also include moving to a multi-zone setup to enhance fault tolerance. Helm charts will be updated for automated deployment of containerized applications. Additionally, NodePools will be reconfigured to include GPU-enabled nodes for AI processing workloads. The new setup will leverage GKE Autopilot mode to ensure optimal resource utilization and autoscaling.', '2024-11-11', 'Improved application performance and scalability', 'immediately', 'high risk', 'cluster scaling', 'under review', 'peer-reviewed', 'planned'),
   ('AB1675', 'This change request proposes a complete overhaul of the internal load balancing mechanism for the web application deployed across AWS Elastic Load Balancers (ELB). The current setup uses a classic load balancer with static routing, which has shown limitations during high-traffic events. The new architecture will involve configuring an Application Load Balancer (ALB) with path-based routing and host-based routing to improve traffic management. AWS WAF (Web Application Firewall) rules will be enhanced to mitigate against OWASP Top 10 vulnerabilities. Additionally, integration with AWS Lambda will enable real-time anomaly detection, with traffic rerouting to spare instances during detected spikes.', '2024-11-11', 'Improved traffic management and security', 'within release window', 'high risk', 'traffic optimization', 'approved', 'uniform agreement', 'planned'),
   ('AB1676', 'This change request details the upgrade of the enterprise Active Directory (AD) infrastructure from Windows Server 2016 to 2022 to support modern authentication protocols like FIDO2 and passwordless login. The update will include the migration of all domain controllers to new virtual machines hosted on VMware vSphere 7.0, with the introduction of a read-only domain controller (RODC) at remote offices for improved security. Group Policy Objects (GPOs) will be reviewed and refactored to comply with the latest security hardening guidelines. Additionally, the AD Federation Services (ADFS) will be upgraded to support integration with Azure AD for hybrid cloud management.', '2024-11-11', 'Enhanced security and modernized authentication', 'within release window', 'moderate risk', 'security enhancement', 'approved', 'peer-reviewed', 'planned'),
   ('AB1677', 'This change request involves the deployment of a multi-region Redis cluster to improve data caching and reduce latency for geographically distributed users. The Redis cluster will be deployed across AWS regions using the ElastiCache service, with automatic failover and cross-region replication to ensure high availability. Each region will host a master Redis node, with slave nodes configured for read scaling. The Redis setup will also be integrated with AWS CloudWatch for real-time performance monitoring and alerting. The implementation will occur during the planned maintenance window, with minimal downtime expected due to pre-warming of the cache nodes.', '2024-11-11', 'Improved data caching and reduced latency', 'within release window', 'moderate risk', 'performance optimization', 'approved', 'no peer review', 'unplanned'),
   ('AB1678', 'This change request proposes the integration of a new AI-based fraud detection module into the transactional systems of the company’s banking platform. The AI model, built with TensorFlow and trained on historical transaction data, will run as a microservice within Kubernetes. The integration will involve the configuration of RESTful API endpoints using Kong API Gateway, which will interface with the transactional database in PostgreSQL. The system will use machine learning techniques such as anomaly detection and supervised learning to flag high-risk transactions in real-time. Full integration will be done in the staging environment for two weeks before going live.', '2024-11-11', 'Enhanced fraud detection capabilities', 'immediately', 'high risk', 'service integration', 'new', 'peer-reviewed', 'planned'),
   ('AB1679', 'This change request involves the transition from traditional on-premises VMware ESXi hosts to a fully managed Kubernetes container orchestration platform in Microsoft Azure Kubernetes Service (AKS). The shift will include replatforming legacy Java-based applications, which are currently running on VMs, to microservices within Kubernetes clusters. ACI (Azure Container Instances) will be used to handle burst workloads, while Azure Blob Storage will be used to manage static data. The deployment will utilize Azure DevOps pipelines for CI/CD automation and will implement Azure Monitor for application performance tracking. The final migration will happen after load testing and validation in the Azure staging environment.', '2024-11-11', 'Improved scalability and cloud-native deployment', 'after approval', 'high risk', 'cloud migration', 'approved', 'manager override', 'unplanned'),
   ('AB1680', 'This change request is focused on the implementation of a centralized logging system using the ELK stack (Elasticsearch, Logstash, Kibana) to collect and analyze application logs across all microservices in the production environment. The ELK cluster will be set up in a dedicated VPC to segregate the traffic from other application components. Logstash will be configured to parse logs from various services, including web servers, application servers, and databases, with Elasticsearch providing the backend for querying logs. Kibana dashboards will be set up for real-time visualization of log data, with automated alerts for anomaly detection. The setup will be tested first in the staging environment before production deployment.', '2024-11-11', 'Centralized logging and real-time monitoring', 'within maintenance window', 'low risk', 'monitoring system installation', 'approved', 'peer-reviewed', 'planned'),
   ('AB1681', 'This change request proposes the configuration of a distributed tracing system using OpenTelemetry and Jaeger to track and optimize the performance of microservices running on AWS Fargate. The Jaeger collector will be deployed in a highly available setup using EC2 instances across multiple availability zones. Application code will be instrumented with OpenTelemetry SDKs for distributed tracing, and traces will be stored in Amazon RDS for historical analysis. The system will also use AWS X-Ray for integration with AWS-native services. Real-time performance metrics will be visible in Grafana dashboards, and alerts will be configured for high latency or failures.', '2024-11-11', 'Improved application performance and observability', 'within release window', 'moderate risk', 'performance optimization', 'under review', 'uniform agreement', 'planned'),
   ('AB1682', 'This change request involves the update of the firewall rules and access control lists (ACLs) in the internal network to tighten security and comply with new regulatory requirements. The firewall will be reconfigured to allow only specific IP ranges from known partners and block all unsolicited inbound traffic. Additionally, VLANs will be configured to segment traffic between different departments, and redundant firewall appliances will be deployed for failover. The changes will be tested in a lab environment before implementation, with logs being forwarded to a Splunk server for real-time analysis.', '2024-11-11', 'Enhanced network security and regulatory compliance', 'immediately', 'high risk', 'security hardening', 'new', 'no peer review', 'unplanned'),
   ('AB1683', 'This change request focuses on upgrading the company’s legacy virtual desktop infrastructure (VDI) from Citrix XenDesktop 7.15 to Citrix Virtual Apps and Desktops 2203. The upgrade will include a complete re-architecting of the Citrix Delivery Controller, Citrix StoreFront, and Citrix Workspace, with the migration of all user profiles and settings to the new environment. Citrix ADC (formerly NetScaler) will also be upgraded to improve session management and load balancing. The new environment will be configured for high availability using Citrix Cloud and integrated with Microsoft Azure AD for single sign-on capabilities. Full user acceptance testing will be conducted in a parallel environment before the final migration.', '2024-11-11', 'Improved virtual desktop experience and scalability', 'within maintenance window', 'moderate risk', 'VDI upgrade', 'approved', 'peer-reviewed', 'planned')

# Create Prompt Function with Python

In [ ]:
def generate_risk_prediction_prompt(cr_data):
    prompt = f"""
            <role>
            You are an experienced dev ops professional deeply knowledgeable on computer systems that support a very large company and the metadata that is captured about change requests.
            A change request is a formal proposal for an alteration to the computer system that you manage.
            As a dev ops expert, you specialize in using the metadata provided about a change request to predict the liklihood of the change request unintentionally destabalizing the computer system.
            You are going to be provided with change request meta data as a json object held between <cr_data> and your job is to provide a prediction score and reasoning behind the risk score in the <output> section. 
            </role>
        
            <task>: Follow these instructions,
            1) Considering the <cr_data> and your <role>, provide a risk score between 0 to 5 of this change request destabalizing the computer system when deployed. do not exhibit a bias toward high risk. base your risk score only on the data you have been provided. if there is not enough information, please indicate this. Output this as [Risk_Score]. Then,
            2) Considering the <cr_data> and your <role>, provide a reasoning for the risk score in as few words as possible while maintaining all detail needed to understand your reasoning. Output this as [Risk_Score_Reason]
            </task>

            <cr_data>
            {cr_data}
            </cr_data>
        
            <Output> 
            produce valid JSON. Absolutely do not include any additional text before or following the JSON. Output should use following <JSON_format>
            </Output>
            
            <JSON_format>
            {{
                "Risk_Score": (A risk score between 0 to 5 of this change request destabalizing the computer system when deployed),
                "Risk_Score_Reason": (A concise resoning for the Risk_Score),
            }}
            </JSON_format>
            """
    return prompt

# Generate Risk Score with Complete
## 🤯 Whoa, check that out 🤯
In a single line of python (22), we conduct our inference. We could swap ANY LLM WE WANT here with exactly 0 overhead. 

In [ ]:
import streamlit as st
from snowflake.cortex import Complete
from snowflake.snowpark.functions import col, call_udf
from snowflake.snowpark.context import get_active_session

#get a session
session = get_active_session()

#replace this with your table
database = 'GEN_AI'
schema = 'PUBLIC'
table = 'ChangeRequests'

#get the data into pandas
#this could be any data but I am using summaries of state of the unions-- replace this with whatever you want.
df = session.table(f"{database}.{schema}.{table}").to_pandas()

#each row's columns are collapsed into a json object. that is easier to pass to the model
#NOTICE: I am calling my generate_prompt function and passing my row as json.
#NOTICE that I am calling complete function here (https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex)
    # VALUE CALLOUT: I could switch out any LLM I have access to (https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability)
df['RISK_ASSESSMENT'] = df.apply(lambda row: Complete('reka-core', generate_risk_prediction_prompt(row.to_json())), axis = 1)

#show us the data
st.dataframe(df['RISK_ASSESSMENT'])

# Make It Relevant with Streamlit in Snowflake
**Note**: I made the below available as a standalone Streamlit in Snowflake app in your environment as well.

In [ ]:
import streamlit as st
import pandas as pd
import json5 as json
from snowflake.cortex import Complete
from snowflake.snowpark.functions import col, call_udf
from snowflake.snowpark.context import get_active_session

#st.image("https://png.pngtree.com/png-clipart/20231007/ourmid/pngtree-beautiful-flying-atlantic-canary-transparent-background-png-image_10196203.png", width = 100)
st.title("Change Request Risk Assessment")
#get a session
session = get_active_session()

def generate_risk_prediction_prompt(cr_data):
    prompt = f"""
            <role>
            You are an experienced dev ops professional deeply knowledgeable on computer systems that support a very large company and the metadata that is captured about change requests.
            A change request is a formal proposal for an alteration to the computer system that you manage.
            As a dev ops expert, you specialize in using the metadata provided about a change request to predict the liklihood of the change request unintentionally destabalizing the computer system.
            You are going to be provided with change request meta data as a json object held between <cr_data> and your job is to provide a prediction score and reasoning behind the risk score in the <output> section. 
            </role>
        
            <task>: Follow these instructions,
            1) Considering the <cr_data> and your <role>, provide a risk score between 0 to 5 of this change request destabalizing the computer system when deployed. do not exhibit a bias toward high risk. base your risk score only on the data you have been provided. if there is not enough information, please indicate this. Output this as [Risk_Score]. Then,
            2) Considering the <cr_data> and your <role>, provide a reasoning for the risk score in as few words as possible while maintaining all detail needed to understand your reasoning. Output this as [Risk_Score_Reason]
            </task>

            <cr_data>
            {cr_data}
            </cr_data>
        
            <Output> 
            produce valid JSON. Absolutely do not include any additional text before or following the JSON. Output should use following <JSON_format>
            </Output>
            
            <JSON_format>
            {{
                "Risk_Score": (A risk score between 0 to 5 of this change request destabalizing the computer system when deployed),
                "Risk_Score_Reason": (A concise resoning for the Risk_Score),
            }}
            </JSON_format>
            """
    return prompt


#replace this with your table
database = 'GEN_AI'
schema = 'PUBLIC'
table = 'ChangeRequests'

#get the data into pandas
cr_df = session.table(f"{database}.{schema}.{table}")

#Get the user's input
cr_request = st.selectbox('Select a change request', cr_df)

#USER INPUT: select model
df_models = pd.DataFrame(['snowflake-arctic', 'mistral-large', 'mistral-large2', 'reka-flash', 'reka-core',
                   'jamba-instruct', 'jamba-1.5-mini', 'jamba-1.5-large', 'mixtral-8x7b', 
                   'llama2-70b-chat', 'llama3-8b', 'llama3-70b', 'llama3.1-8b', 'llama3.1-70b',
                   'llama3.1-405b', 'llama3.2-1b', 'llama3.2-3b', 'mistral-7b', 'gemma-7b'], 
                  columns=['Model Name'])

user_input_model = st.selectbox("Select Model", df_models, key="CS_model_select_box")


df = session.table(f"{database}.{schema}.{table}").filter(col("CHANGENUMBER")  == cr_request).to_pandas()

if st.button("Run", type="primary"):
    #each row's columns are collapsed into a json object. that is easier to pass to the model
    #NOTICE: I am calling my generate_prompt function and passing my row as json.
    #NOTICE that I am calling complete function here (https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex)
        # VALUE CALLOUT: I could switch out any LLM I have access to (https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability)
    df['RISK_ASSESSMENT'] = df.apply(lambda row: Complete(user_input_model, generate_risk_prediction_prompt(row.to_json())), axis = 1)

    #show us the data
    
    try:
        json = json.loads(df.at[0, 'RISK_ASSESSMENT'])
    except ValueError:
        # Code to handle ValueError (if the input is not an integer)
        st.write(f"Whoops! {user_input_model}, the model you selected, did not produce valid output. Please select another model")
    else: 
        st.metric(label="Risk Score", value=json["Risk_Score"])
        st.write(json["Risk_Score_Reason"])
        st.subheader("Raw json")
        st.json(json)